<a href="https://colab.research.google.com/github/klesment/PopProj/blob/main/M%C3%A4rkmiku_PopulationProjection_ipynb_koopia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import gamma
import requests
import re
import io
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import interact, interact_manual, interactive, fixed, IntSlider, FloatSlider, HBox, Layout, VBox
import cufflinks as cf
cf.go_offline()

from IPython.display import Markdown as md
%matplotlib inline

In [22]:
# Data files and constant values

# age specific fertility rates
URL = 'https://raw.githubusercontent.com/klesment/PopProj/main/ESTasfrRR.txt'
s = requests.get(URL).content
asfr = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='\s+', header = 2)
asfr = asfr.apply(pd.to_numeric, errors = 'coerce')

# life table data
URL = 'https://raw.githubusercontent.com/klesment/PopProj/main/LT.txt'
s = requests.get(URL).content
lt = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='\s+', header = 1)
lt.loc[lt['Age'] == '110+', 'Age'] = 110 # open upper age interval to 110
lt = lt.apply(pd.to_numeric)
lt.loc[lt['qx'] == 0, 'qx'] = np.nan # zeros to NaN

# population counts
URL = 'https://raw.githubusercontent.com/klesment/PopProj/main/Population.txt'
s = requests.get(URL).content
pop = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='\s+', header = 1)
pop.loc[pop['Age'] == '110+', 'Age'] = 110 # open upper age interval to 110
pop = pop.apply(pd.to_numeric)

# mean age at birth in 2019
mab = 30.62

# standard deviation of mean age at birth in 2019
sd_mab = 5.62

In [23]:
# Constructing and populating the Leslie matrix

# starting values
base_year = 2019

# life table
lt19 = lt[lt['Year']==base_year]

# population structure
pop19 = pop[pop['Year']==base_year]
N0 = pop19.Total[0:110].tolist()

# ASFR
asfr19 = asfr[asfr['Year']==base_year]

# Matrix elements
# Probability of survival at each age from life table, goes to matrix subdiagonal
SUBD = lt19.Lx.iloc[1:].values/lt19.Lx.iloc[0:110].values
SUBD[-1:] = 0
Tx = lt19['Tx']

# Age-specific fertility rates, values for ages 12-54
Fx = [0]*12 + asfr19.ASFR.values.tolist() + [0]*55
k = (1/(1+1.05))*(lt19.Lx.iloc[0]/200000)
R1 = (k*(np.array(Fx[0:110]) + np.array(Fx[1:111]) * np.array(SUBD[0:110])))

# matrix
L = np.zeros((109, 110))

np.fill_diagonal(L, SUBD[0:109])

L = np.vstack((R1,L))

L[109,109] = Tx.values[110]/Tx.values[109]


# Projection function

def proj(m, t, n0):
    Nproj = []

    for i in range(0, t, 1):
        if i == 0:
            project = m
            Nproj = np.dot(project, n0)

        if i > 0:
            project = np.dot(project, m)
            Nproj = np.dot(project, n0)

    return Nproj

In [24]:
# Gamma model function

def asfr_gamma(aa):
    f_x_out = []

  # parameters
    a1 = aa['tfr']
    a2 = aa['mab']/(aa['sd_mab']**2)
    a3 = (aa['mab']/aa['sd_mab'])**2

  # age range
    ages = list(range(12,50,1))

    for age in ages:
        v = (1/gamma(a3))*a1*(a2**a3)*(age**(a3-1))*np.exp(-a2*age)
        f_x_out.append(v)

  # 1-D array
    return np.array(f_x_out).ravel()

In [25]:
# Define new functions

# dynamic projection function

def project_dyn(lmat, pop, per):
    '''
    lmat - yearly L matrix; pop - initial population structure; per - period to be projected
    '''

    N_0 = pop

    for i in range(1, per, 1):
        N_x = proj(lmat[(i-1)], 1, N_0)
        N_0 = N_x

    return N_0


# Leslie matrix function

def leslie(fert):
    '''
    fert - ASFR vector;
    mort - life table dataframe
    'lt19' data frame must be present
    '''
    mort = lt19

    SUBD = mort.Lx.iloc[1:].values/mort.Lx.iloc[0:110].values
    SUBD[-1:] = 0

    Tx = mort['Tx']

    Fx = [0]*12 + fert.values.tolist() + [0]*61
    k = (1/(1+1.05))*(mort.Lx.iloc[0]/200000)
    R1 = (k*(np.array(Fx[0:110]) + np.array(Fx[1:111]) * np.array(SUBD[0:110])))

    L = np.zeros((109, 110))
    np.fill_diagonal(L, SUBD[0:109])
    L = np.vstack((R1,L))
    L[109,109] = Tx.values[110]/Tx.values[109]

    return L

In [26]:
# TFR change ramp function

def ramp_fun(TFR_chng,speed,pr_per):
    a,b,c = TFR_chng,5,speed

    x = np.linspace(0,1,pr_per)
    y = a*np.exp(-b*np.exp(-c*x))
    y2 = [y[-1]]*0
    return np.add([*y,*y2],1)

In [27]:
# @title
plt.rcParams['figure.figsize'] = [14, 8]

def dynamic_graph(TFR_Change, Ramp, MAB_end, sdMAB_end, Years):
    mab_stop,sd_mab_stop,period = MAB_end,sdMAB_end,Years

  # fixed starting values from source data frames
    tfr_start,mab_start,sd_mab_start = round(sum(asfr['ASFR'][asfr['Year']==base_year]),2),mab,sd_mab

    if period==0:
        period += 1

  # user input values
    d = pd.DataFrame(data={
        'tfr': np.linspace(tfr_start, tfr_start, period),
        'mab': np.linspace(mab_start, mab_stop, period),
        'sd_mab': np.linspace(sd_mab_start, sd_mab_stop, period)
     })

    d['tfr'] = np.multiply(d['tfr'], ramp_fun(TFR_Change,Ramp,period))

    if period==0:
        tfr_last = round(d['tfr'][period],2)
    if period>0:
        tfr_last = round(d['tfr'][period-1],2)

  # age-specific fertility rates from the gamma function
    F = []
    F = np.array(d.apply(asfr_gamma, axis=1).tolist())

    D = pd.DataFrame(data=F[0:,0:])

  # Leslie matrices as lists
    LL = np.array(D.apply(leslie, axis=1))

  # projection function
    out = project_dyn(LL, N0, period)

    p_size_start,p_size_end = round(sum(pop['Total'][pop['Year']==base_year])/1000_000,3),round(sum(out)/1000000, 3)

    plt.clf()

    p = sns.lineplot(x=range(1,111,1), y=out, linewidth=2.5)
    if period<151:
        plt.plot([period,period], [5000,15000], color='red', ls='dotted')
        plt.ylim([0, 25_000])
        plt.autoscale(False)
        p.set(xlabel='1-year age group', ylabel='Count')
        p.annotate(f"{base_year + len(LL)}: TFR={tfr_last} mAB={mab_stop} N={p_size_end} Mio.", xy=(0,21_500), size=16)
        p.annotate(f"{base_year}: TFR = {tfr_start} mAB={mab_start} N={p_size_start} Mio.", xy=(0,23_000), size=16)
        p.annotate(f"Newborns last year: {round(out[0])}", xy=(80,22_000), color="red", size=16)
        p.annotate(f"Pop.change to %: {round(p_size_end/p_size_start*100,1)}", xy=(80,20_000), color="red", size=16)

    plt.show()

widget = interactive(dynamic_graph,
                TFR_Change = FloatSlider(min=-.5, max=.5, step=.1, value=-0.3, orientation='horizontal'),
                Ramp = FloatSlider(min=5, max=8, step=.1, value=5, orientation='horizontal'),
                MAB_end=IntSlider(min=18, max=36, step=1, value=30, orientation='horizontal'),
                sdMAB_end=FloatSlider(min=5, max=6, step=.1, value=5.5, orientation='horizontal'),
                Years=IntSlider(min=0, max=150, step=5, value=5, orientation='horizontal'))

controls = HBox(widget.children[:-1], layout = Layout(flex_flow='row wrap'))
output = widget.children[-1]
display(VBox([controls, output]))
widget.update()